In [2]:
import json, urllib, csv, locale, collections
## import glob, os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time, random, requests

In [3]:
drinkByName = dict()
"""
with open('../app/static/drinks.json', 'r') as infile:  
    jsonData = json.load(infile)
    drinklst = jsonData['drinks']

for drink in drinklst:
    drinkByName[drink['name']] = drink
"""

"\nwith open('../app/static/drinks.json', 'r') as infile:  \n    jsonData = json.load(infile)\n    drinklst = jsonData['drinks']\n\nfor drink in drinklst:\n    drinkByName[drink['name']] = drink\n"

In [4]:
cookies = dict(cookie="%7B%22acknowledge%22%3Atrue%2C%22when%22%3A1556481723%2C%22REMOTE_ADDR%22%3A%22128.84.127.236%22%2C%22INIT_REMOTE_ADDR%22%3A%22192.168.2.168%22%2C%22INIT_FWD_ADDR%22%3A%22128.84.127.236%2C+70.132.21.99%22%2C%22location%22%3A%7B%22country_code%22%3A%22US%22%2C%22country_name%22%3A%22United+States%22%2C%22gdpr%22%3Afalse%7D%7D")

In [12]:
def scrape_drink(url):
    drink = dict()
    
    session = requests.Session()
    r = session.post(url=url, cookies=cookies)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    name = soup.find('h1', attrs={'itemprop':'name'})
    drink['name'] = name.text.strip()
    
    #flavor = soup.find('div', attrs={'class':'drink-properties row'})
    #flavor = flavor.find_all('p')[1]
    #drink['flavors'] = [a.strip() for a in flavor.text.split(',')]
    
    img = soup.find('div', attrs={'class':'row image-row pad-bottom'})
    img = img.find('img', src=True)
    drink['src'] = img['src'][2:]
    
    drink['ingredients'] = list()
    
    ingredients = soup.find_all('div', attrs={'class':'row x-recipe-unit'})
    for ing in ingredients:
        oz = ing.find('div', attrs={'class':'oz-value'})
        i = ing.find('div', attrs={'class':'col-xs-9 x-recipe-ingredient'})
        
        if i.find('a') is not None:
            i = i.find('a')
            
        drink['ingredients'].append((oz.text.strip(), i.text.strip()))
    
    #data = soup.find('div', attrs={'class':'col-xs-12 text-center x-recipe-flavor recipe-link'})
    data = soup.find('div', attrs={'class':'row hidden-print pad-bottom'})
    data = data.find_all('div', attrs={'class':'row'})
    for d in data:
        if d.find('div', attrs={'class':'col-xs-12 text-center x-recipe-flavor recipe-link'}) is not None:
            data = d
            break
            
    flavors = d.find('div', attrs={'class':'col-xs-12 text-center x-recipe-flavor recipe-link'})
    if flavors is not None:
        drink['flavors'] = [a.text.strip() for a in flavors.find_all('a')]
    
    rows = data.find('div', attrs={'class':'row'})
    
    ctype = rows.find('div', attrs={'class':'col-xs-7 x-recipe-type'})
    if ctype is not None:
        drink['type'] = ctype.text.strip()
    
    hours = rows.find('div', attrs={'class':'col-xs-7 x-recipe-hours'})
    if hours is not None:
        drink['hours'] = [a.text.strip() for a in hours.find_all('a')]
    
    
    drink['tags'] = list()
    themes = rows.find('div', attrs={'class':'col-xs-7 x-recipe-themes'})
    if themes is not None:
        drink['tags'] += [a.text.strip() for a in themes.find_all('a')]
    
        
    occ = rows.find('div', attrs={'class':'col-xs-7 x-recipe-occasions'})
    if occ is not None:
        drink['tags'] += [a.text.strip() for a in occ.find_all('a')]
    return drink

print(scrape_drink("https://www.liquor.com/recipes/audrey-saunders-tom-and-jerry/"))

{'src': 'cdn.liquor.com/wp-content/uploads/2018/12/10122259/tom-and-jerry-720x720-recipe.jpg', 'type': 'Classics', 'name': "Audrey Saunders' Tom & Jerry", 'tags': ['Holiday Season', 'Christmas', 'Holiday party'], 'flavors': ['Creamy', 'Sweet'], 'hours': ['Evening'], 'ingredients': [('2\xa0oz', 'Tom & Jerry batter*'), ('1\xa0oz', 'Plantation Barbados rum'), ('1\xa0oz', 'Pierre Ferrand Ambre cognac'), ('6\xa0oz', 'Milk, heated')]}


In [9]:
urls = dict()

def scrape_urls():
    i = 1
    while True:
        
        page_url = "https://www.liquor.com/page/"+str(i)+"/?post_type=recipe&s#038;s"
        
        session = requests.Session()
        r = session.post(page_url, cookies=cookies)

        soup = BeautifulSoup(r.text, 'html.parser')
        
        soup = soup.find('div', attrs={'class':'container archive-content'})

        drinks = soup.find_all('div', attrs={'class':'item card card-type-recipe '})

        for d in drinks:
            u = d.find('a', href=True)
            u = u['href']
            
            urls[u] = True
            
        print(len(urls))
        i += 1
        r.connection.close()
        
scrape_urls()
print(len(urls))

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2188


AttributeError: 'NoneType' object has no attribute 'find_all'

In [29]:
def scrape_pages():
    for url in urls.keys():
        if urls[url] != True:
            continue
            
        time.sleep(random.randint(2,5))
            
        try:
            d = scrape_drink(url)
            drinkByName[d['name']] = d
            print(d['name'])
            urls[url] = False
            
        except AttributeError:
            print("FAILED ON "+str(url))

scrape_pages()

FAILED ON https://www.liquor.com/recipes/pea-puree/
FAILED ON https://www.liquor.com/recipes/grilled-lamb-tacos/
FAILED ON https://www.liquor.com/recipes/roasted-poblano-infused-tequila/
FAILED ON https://www.liquor.com/recipes/john-the-baptist/
FAILED ON https://www.liquor.com/recipes/portobello-fajita-tacos/
FAILED ON https://www.liquor.com/recipes/salmon-tacos-with-mango-salsa/
FAILED ON https://www.liquor.com/recipes/spicy-shredded-chicken-tacos/
FAILED ON https://www.liquor.com/recipes/savory-eyeballs/
FAILED ON https://www.liquor.com/recipes/godfather-101/
FAILED ON https://www.liquor.com/recipes/shredded-beef-tacos/
FAILED ON https://www.liquor.com/recipes/oatmeal-infused-karlssons-gold-vodka/
FAILED ON https://www.liquor.com/recipes/cazadores-grilled-shrimp-tacos-corn-salsa/
FAILED ON https://www.liquor.com/recipes/vanilla-simple-syrup/
FAILED ON https://www.liquor.com/recipes/cazadores-skirt-steak-tacos/
FAILED ON https://www.liquor.com/recipes/cazadores-pork-pineapple-tacos/


Garrick Club Punch
Grassland Wallow
Pear & Elderflower Collins
Watermelon Margarita
Montauk
Between the Sheets
Flying Elvis
Cinnamon Sparkle
El Guapo
Room Temperature Martini No. 3
Ranye West
Blanco & Tonic
The Coco Chanel Martini
Stoli Bloody Mary
Bacardí El Espresso de San Juan
Lawless Liter
Ten French 75
Shrub Julep
Tipperary
NOLET’S Strawberry Mint Punch
Snickertini
Kew Garden
The Orkney Chapel
Mink Coat and No Manners
10 Cane Hemingway Daiquiri
Camus Fresh
Deshler
Dulce Y Picante
The Celfie
Finlandia Oaks Lily
Trash Tiki Citrus Stock
Trato Hecho
Tequila Furnace
Lynx
Two-Step Easy Brandied Cherries
Bronx
Milagro Ancho Paloma
Hydrate
Elyx Fennel Countdown
The Japanese
Concord Grape Margarita
Cadbury Creme Egg
Green Derby
Nassau Valley
Mezcal Negroni
Toronto
Green Eyes
Bloody Maria
Honey Berry Sling
Man o' War
Sans-Culottes
White Hot American Summer
Hennessy Privilège Sidecar
My Spicy Valentine
Pardy Hardy
All Fired Up
Belvedere Polish 75
Whiskey in a Castle
Sour Grapes
The Duchess o

In [30]:
print(len(drinkByName))
print(drinkByName['Basil Hayden’s Kentucky Buck'])

2092
{'src': 'cdn.liquor.com/wp-content/uploads/2011/01/basil-haydens-irish-buck.jpg', 'type': 'Modern Classics', 'name': 'Basil Hayden’s Kentucky Buck', 'tags': [], 'flavors': ['Fruity/Citrus-forward', 'Sour', 'Sweet'], 'hours': ['Afternoon', 'Dinner/Paired with food', 'Evening'], 'ingredients': [('1 1⁄4\xa0oz', "Basil Hayden\\'s Bourbon"), ('1⁄2\xa0oz', 'DeKuyper Triple Sec'), ('1\xa0splash', 'Orange juice'), ('1\xa0splash', 'Cranberry juice'), ('1\xa0splash', 'Simple syrup')]}


In [31]:
jsondata = {}
jsondata['drinks'] = []
for drink in drinkByName.values():
    if 'type' not in drink:
        drink['type'] =''
    if 'hours' not in drink:
        drink['hours'] =[]
    
    jsondata['drinks'].append({
        'name' : drink['name'],
        'ingredients' : drink['ingredients'],
        'flavors' : drink['flavors'],
        'src' : drink['src'],
        'type' : drink['type'],
        'hours' : drink['hours'],
        'tags' : drink['tags']
    })

In [32]:
print(len(jsondata['drinks']))

2092


In [33]:
with open('./tag-data.json', 'w') as outfile:  
    json.dump(jsondata, outfile)

In [24]:
print(urls)

{'https://www.liquor.com/recipes/appleton-estate-from-the-cradle-to-the-grave/': False, 'https://www.liquor.com/recipes/basil-haydens-kentucky-buck/': False, 'https://www.liquor.com/recipes/the-libertea/': False, 'https://www.liquor.com/recipes/el-bicentenario/': False, 'https://www.liquor.com/recipes/blackberry-wine-slushy/': False, 'https://www.liquor.com/recipes/provence-cooler/': False, 'https://www.liquor.com/recipes/bacardi-superior-mai-tai/': False, 'https://www.liquor.com/recipes/iced-caramel-coffee/': False, 'https://www.liquor.com/recipes/voodoo-eyes/': False, 'https://www.liquor.com/recipes/start-me-up/': False, 'https://www.liquor.com/recipes/perfect-pear/': False, 'https://www.liquor.com/recipes/nolets-bramble/': False, 'https://www.liquor.com/recipes/starry-night/': False, 'https://www.liquor.com/recipes/tamarind-margarita/': False, 'https://www.liquor.com/recipes/tequila-daisy/': False, 'https://www.liquor.com/recipes/clairin-regal-sour/': False, 'https://www.liquor.com/